In [1]:
# user-friendly print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np

from collections import defaultdict
from crystallus import pbc_all_distances, CrystalGenerator, WyckoffCfgGenerator
from pymatgen import Structure
from pymatgen.analysis.structure_analyzer import SpacegroupAnalyzer

Re-generate with same wyckoff positions

In [ ]:
struct = Structure.from_file('178_224.cif')
analyzer = SpacegroupAnalyzer(struct)
sym_info = analyzer.get_symmetry_dataset()

In [ ]:
species = [str(s) for s in struct.species]
wyckoffs = sym_info['wyckoffs']

cfg = defaultdict(list)

for e, wy in zip(species, wyckoffs):
    if wy not in cfg[e]:
        cfg[e].append(wy)
        
cfg

In [ ]:
cg = CrystalGenerator(
    spacegroup_num=sym_info['number'],
    estimated_volume=922,
    estimated_variance=15,
    min_distance_tolerance=0.1,
    max_recurrent=5_000,
    verbose=True)
cg

In [5]:
cg.gen_one(**cfg)

ValueError: crystal generator error: `reached the max recurrent number, please try to set `estimated_volume` and/or `min_distance_tolerance` bigger. (in crystal structure generation)`

Test `Ag32Ge4S24`.

* Spacegroup 33
* Wyckoff positions:
    (x,y,z) (-x,-y,z+1/2) (x+1/2,-y+1/2,z) (-x+1/2,y+1/2,z+1/2)
* Ground truth:
    {Ag: 4a * 8, Ge: 4a, S: 4a * 5}

In [6]:
wg = WyckoffCfgGenerator(Ag=32, Ge=4, S=24)
wg

WyckoffCfgGenerator(            
    max_recurrent=1000,            
    n_jobs=-1            
    composition={'Ag': 32, 'Ge': 4, 'S': 24}            
)

In [7]:
cfgs = wg.gen_many(1000, 33)
cfgs

[{'Ag': ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
  'Ge': ['a'],
  'S': ['a', 'a', 'a', 'a', 'a', 'a']}]

Because the space group 33 only has one position set, even if we want to find various Wyckoff configurations, the result has one configuration.

In [8]:
cg = CrystalGenerator(
    spacegroup_num=33,
    estimated_volume=1168.454590,
    estimated_variance=15,
    min_distance_tolerance=0.1,
    max_recurrent=50_000,
    verbose=True)
cg

CrystalGenerator(            
    spacegroup_num=33,            
    estimated_volume=1168.45459,            
    estimated_variance=15,            
    min_distance_tolerance=0.1,            
    angle_range=(30.0, 150.0),            
    angle_tolerance=20.0,            
    max_recurrent=50000,            
    n_jobs=-1            
)

In [14]:
%%time

raw_ss = cg.gen_many(3, cfgs[0], check_distance=False)

CPU times: user 745 µs, sys: 832 µs, total: 1.58 ms
Wall time: 362 µs


In [16]:
for i, raw_s in enumerate(raw_ss):
    Structure(lattice=raw_s['lattice'], species=raw_s['species'], coords=raw_s['coords']).to('cif', filename=f'Ag32Ge4S24/{i}.cif')